In [1]:
import json
import torch
import pickle
import os
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm


from data.dataset import NERDataset
from models.networks import GlobalContextualDeepTransition
from scripts.beamsearch import BeamSearcher

In [11]:
sourceName = 'data/conll03/eng.train.src'
targetName = 'data/conll03/eng.train.trg'
gloveFile = 'data/conll03/trimmed.300d.Cased.txt'
symbFile = 'data/conll03/sym.glove'
testSrc = 'data/conll03/eng.testa.src'
testTrg = 'data/conll03/eng.testa.trg'
predFile = 'preds/' + os.path.basename(testTrg) + '.out'

data = NERDataset(sourceName, targetName, gloveFile, symbFile)
data.readTestFile(testSrc, testTrg)
loader = data.getLoader(4096, shuffle=False)

In [3]:
prevCheckpointPath = 'lightning_logs/rasputin_217.ckpt'

with open('config.json', 'r') as file:
    kwargs = json.load(file)
    
model = GlobalContextualDeepTransition.load_from_checkpoint(prevCheckpointPath, **kwargs)
model = model.eval().cuda()

In [4]:
tester = BeamSearcher(beamSize=4, model=model)

In [5]:
out = []
for batch in tqdm(loader):
    batch = (w.cuda() for w in batch)
    preds = tester(batch)
    out.extend(preds)

In [12]:
tester.writePreds(out, data.tags, predFile, testSrc)

In [13]:
def countWords(filename):
    count = 0
    with open(filename, 'r') as file:
        for line in file:
            count += len(line.strip().split())
    return count

In [14]:
print(countWords(predFile), countWords(testSrc))

51578 51578


In [15]:
result = 'results/'+ os.path.basename(prevCheckpointPath) + '.' + os.path.basename(predFile)
print('Saving file at ', result)
tester.getResultFile(testSrc, predFile, testTrg, result)

Saving file at  results/rasputin_217.ckpt.eng.testa.trg.out
Predictions with bad length: 0
